In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Read the CSV file
df = pd.read_csv('./data/BTCUSDT_1m.csv')

# Display summary statistics
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df = df.sort_values(by='open_time', ascending=True).reset_index(drop=True)
df.describe()


In [ ]:
# Get memory usage of the dataframe
memory_usage = df.memory_usage(deep=True)

# Convert to MB for better readability
memory_usage_mb = memory_usage.sum() / 1024 / 1024

print("\nMemory usage breakdown by column (bytes):")
for column, usage in memory_usage.items():
    print(f"{column}: {usage:,} bytes")
    
print(f"\nTotal memory usage: {memory_usage_mb:.2f} MB")


In [ ]:
df.columns

In [ ]:
# Create a dictionary mapping old column names to new ones with '1m_' prefix
rename_dict = {
    'open_time': '1m_open_time',
    'close_time': '1m_close_time', 
    'open_px': '1m_open_px',
    'high_px': '1m_high_px',
    'low_px': '1m_low_px',
    'close_px': '1m_close_px',
    'number_of_trades': '1m_number_of_trades',
    'base_asset_volume': '1m_base_asset_volume',
    'quote_asset_volume': '1m_quote_asset_volume',
    'taker_buy_base_asset_volume': '1m_taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume': '1m_taker_buy_quote_asset_volume'
}

# Rename the columns
df = df.rename(columns=rename_dict)

# Display the new column names
print("New column names:")
print(df.columns)


In [ ]:
df[['1m_open_time', '1m_close_time', '1m_open_px', '1m_high_px', '1m_low_px', '1m_close_px']].head(10)

In [ ]:
# Create 5-minute aggregations
# First create a group index (0-4) that repeats every 5 rows
df['group_idx'] = df.index % 5

# Create is_5m_closed column - True for last entry in each group
df['is_5m_closed'] = df['group_idx'] == 4

# Group by integer division of index to get 5-minute groups
groups = df.groupby(df.index // 5)

# Create 5m columns - using cummax/cummin within groups for high/low prices
df['5m_open_time'] = groups['1m_open_time'].transform('first')
df['5m_close_time'] = groups['1m_close_time'].transform('last') 
df['5m_open_px'] = groups['1m_open_px'].transform('first')
df['5m_high_px'] = groups['1m_high_px'].transform('cummax')
df['5m_low_px'] = groups['1m_low_px'].transform('cummin')
df['5m_close_px'] = df['1m_close_px']

# Drop temporary grouping column
df = df.drop('group_idx', axis=1)

# Display first few rows to verify
df[['5m_open_time', '5m_close_time', '5m_open_px', '5m_high_px', '5m_low_px', '5m_close_px', 'is_5m_closed']].head(10)


In [ ]:
# Create 15-minute aggregations from 1m data
df['group_idx'] = df.index % 15

# Create is_15m_closed column - True for last entry in each group
df['is_15m_closed'] = df['group_idx'] == 14

# Group by integer division of index to get 5-minute groups
groups = df.groupby(df.index // 15)

# Create 15m columns - using cummax/cummin within groups for high/low prices
df['15m_open_time'] = groups['1m_open_time'].transform('first')
df['15m_close_time'] = groups['1m_close_time'].transform('last') 
df['15m_open_px'] = groups['1m_open_px'].transform('first')
df['15m_high_px'] = groups['1m_high_px'].transform('cummax')
df['15m_low_px'] = groups['1m_low_px'].transform('cummin')
df['15m_close_px'] = df['1m_close_px']

# Drop temporary grouping column
df = df.drop('group_idx', axis=1)

# Display first few rows to verify
df[['15m_open_time', '15m_close_time', '15m_open_px', '15m_high_px', '15m_low_px', '15m_close_px', 'is_15m_closed']].head(15)



In [ ]:
# Create 1-hour aggregations from 1m data
df['group_idx'] = df.index % 60

# Create is_5m_closed column - True for last entry in each group
df['is_1h_closed'] = df['group_idx'] == 59

# Group by integer division of index to get 5-minute groups
groups = df.groupby(df.index // 60)

# Create 5m columns - using cummax/cummin within groups for high/low prices
df['1h_open_time'] = groups['1m_open_time'].transform('first')
df['1h_close_time'] = groups['1m_close_time'].transform('last') 
df['1h_open_px'] = groups['1m_open_px'].transform('first')
df['1h_high_px'] = groups['1m_high_px'].transform('cummax')
df['1h_low_px'] = groups['1m_low_px'].transform('cummin')
df['1h_close_px'] = df['1m_close_px']

# Drop temporary grouping column
df = df.drop('group_idx', axis=1)

# Display first few rows to verify
df[['1h_open_time', '1h_close_time', '1h_open_px', '1h_high_px', '1h_low_px', '1h_close_px', 'is_1h_closed']].head(15)

In [ ]:
# Create 4-hour aggregations from 1m data
df['group_idx'] = df.index % 240

# Create is_4h_closed column - True for last entry in each group
df['is_4h_closed'] = df['group_idx'] == 239

groups = df.groupby(df.index // 240)

df['4h_open_time'] = groups['1m_open_time'].transform('first')
df['4h_close_time'] = groups['1m_close_time'].transform('last') 
df['4h_open_px'] = groups['1m_open_px'].transform('first')
df['4h_high_px'] = groups['1m_high_px'].transform('cummax')
df['4h_low_px'] = groups['1m_low_px'].transform('cummin')
df['4h_close_px'] = df['1m_close_px']

# Drop temporary grouping column
df = df.drop('group_idx', axis=1)

# Display first few rows to verify
df[['4h_open_time', '4h_close_time', '4h_open_px', '4h_high_px', '4h_low_px', '4h_close_px', 'is_4h_closed']].head(15)

In [ ]:
# Create 1-day aggregations from 1m data
df['group_idx'] = df.index % 1440

# Create is_1d_closed column - True for last entry in each group
df['is_1d_closed'] = df['group_idx'] == 1439

groups = df.groupby(df.index // 1440)

df['1d_open_time'] = groups['1m_open_time'].transform('first')
df['1d_close_time'] = groups['1m_close_time'].transform('last') 
df['1d_open_px'] = groups['1m_open_px'].transform('first')
df['1d_high_px'] = groups['1m_high_px'].transform('cummax')
df['1d_low_px'] = groups['1m_low_px'].transform('cummin')
df['1d_close_px'] = df['1m_close_px']

df = df.drop('group_idx', axis=1)

df[['1d_open_time', '1d_close_time', '1d_open_px', '1d_high_px', '1d_low_px', '1d_close_px', 'is_1d_closed']].head(15)


In [ ]:
# Strategy 1, Buy on 0:00, Sell on 12:00
df["signal_1"] = 0
df.loc[df["1m_open_time"] % 86400000 == 0, "signal_1"] = 1
df.loc[df["1m_open_time"] % 86400000 == 43200000, "signal_1"] = -1
# Vectorized calculation of asset positions
df["asset_1_usdt"] = 1000
df["asset_1_btc"] = 0

df["signal_1"].describe()

In [ ]:

n = len(df)
usdt_list = [0] * n
btc_list = [0] * n
portfolio_list = [0] * n

usdt_list[0] = 1000
btc_list[0] = 0
portfolio_list[0] = 1000


n_trade_usdt = 500

for i in range(len(df) - 1):
    n_btc = btc_list[i]
    n_usdt = usdt_list[i]
    price = df["1m_close_px"].iloc[i]
    signal = df["signal_1"].iloc[i]

    if signal == 1:
        n_usdt = n_usdt - n_trade_usdt
        n_btc = n_btc + n_trade_usdt / price
    elif signal == -1:
        n_usdt = n_usdt + n_trade_usdt
        n_btc = n_btc - n_trade_usdt / price
    elif signal == 0:
        pass
    else:
        raise ValueError("Invalid signal")
    
    usdt_list[i + 1] = n_usdt
    btc_list[i + 1] = n_btc
    
df["btc_1"] = btc_list
df["usdt_1"] = usdt_list
df["portfolio_1_open_px"] = df["btc_1"] * df["1m_open_px"] + df["usdt_1"]
df["portfolio_1_high_px"] = df["btc_1"] * df["1m_high_px"] + df["usdt_1"]
df["portfolio_1_low_px"] = df["btc_1"] * df["1m_low_px"] + df["usdt_1"]
df["portfolio_1_close_px"] = df["btc_1"] * df["1m_close_px"] + df["usdt_1"]


In [ ]:
# Create 1-day aggregations from 1m data
df['group_idx'] = df.index % 1440

groups = df.groupby(df.index // 1440)

# Aggregate portfolio values to daily level
df['1d_portfolio_1_open'] = groups['portfolio_1_open_px'].transform('first')
df['1d_portfolio_1_high'] = groups['portfolio_1_high_px'].transform('max')
df['1d_portfolio_1_low'] = groups['portfolio_1_low_px'].transform('min')
df['1d_portfolio_1_close'] = groups['portfolio_1_close_px'].transform('last')

df = df.drop('group_idx', axis=1)

df[['1d_portfolio_1_open', '1d_portfolio_1_high', '1d_portfolio_1_low', '1d_portfolio_1_close']].describe()

In [ ]:
# Select January 2022 data (first month in the dataset based on context)
start_time = 1640995200000  # Jan 1, 2022
end_time = 1643673600000   # Jan 31, 2022
df_portfolio = df[['1d_open_time', '1d_close_time', '1d_portfolio_1_open', '1d_portfolio_1_high', '1d_portfolio_1_low', '1d_portfolio_1_close', 'is_1d_closed']]

df_portfolio = df_portfolio[df_portfolio['is_1d_closed'] == True]
df_portfolio
# Filter data for January
jan_portfolio = df_portfolio[(df_portfolio['1d_open_time'] >= start_time) & (df_portfolio['1d_open_time'] < end_time)]
# 
# 
# # Plot candlestick chart
import mplfinance as mpf

# Create a pandas DataFrame with proper datetime index
jan_portfolio_df = jan_portfolio.copy()
jan_portfolio_df.index = pd.date_range(start='2022-01-01', periods=len(jan_portfolio_df), freq='D')
jan_portfolio_df = jan_portfolio_df.rename(columns={
    '1d_open_time': 'Date',
    '1d_portfolio_1_open': 'Open',
    '1d_portfolio_1_high': 'High',
    '1d_portfolio_1_low': 'Low',
    '1d_portfolio_1_close': 'Close'
})

# Plot using mplfinance
mpf.plot(jan_portfolio_df, 
         type='candle',
         title='Portfolio Performance - January 2022',
         ylabel='Portfolio Value (USDT)',
         volume=False,
         style='yahoo')


In [ ]:
# Select January 2022 data (first month in the dataset based on context)
start_time = 1640995200000  # Jan 1, 2022
end_time = 1643673600000   # Jan 31, 2022
df_btc = df[['1d_open_time', '1d_close_time', '1d_open_px', '1d_high_px', '1d_low_px', '1d_close_px', 'is_1d_closed']]

df_btc = df_btc[df_btc['is_1d_closed'] == True]
# Filter data for January
jan_btc = df_btc[(df_btc['1d_open_time'] >= start_time) & (df_portfolio['1d_open_time'] < end_time)]
# 
# 
# # Plot candlestick chart
import mplfinance as mpf

# Create a pandas DataFrame with proper datetime index
jan_btc_df = jan_btc.copy()
jan_btc_df.index = pd.date_range(start='2022-01-01', periods=len(jan_btc_df), freq='D')
jan_btc_df = jan_btc_df.rename(columns={
    '1d_open_time': 'Date',
    '1d_open_px': 'Open',
    '1d_high_px': 'High',
    '1d_low_px': 'Low',
    '1d_close_px': 'Close'
})
jan_btc_df.head()

# Plot using mplfinance
mpf.plot(jan_btc_df, 
         type='candle',
         title='BTC Performance - January 2022',
         ylabel='BTC Value (USDT)',
         volume=False,
         style='yahoo')

In [ ]:
# Get January 2022 start and end timestamps
jan_2022_start = pd.Timestamp('2022-01-01').timestamp() * 1000  # Convert to milliseconds
jan_2022_end = pd.Timestamp('2022-02-01').timestamp() * 1000  # Convert to milliseconds


# Filter data for January 2022
jan_df = df[(df['1m_open_time'] >= jan_2022_start) & (df['1m_open_time'] < jan_2022_end)]
# Calculate monthly simple return using first and last close prices
first_close = jan_df['portfolio_1_close_px'].iloc[0]
last_close = jan_df['portfolio_1_close_px'].iloc[-1]
simple_return = (last_close - first_close) / first_close
# Calculate monthly volatility using daily returns
minutely_returns = jan_df['portfolio_1_close_px'].pct_change()
monthly_volatility = minutely_returns.std() * np.sqrt(len(jan_df))  # Monthly volatility
# Calculate maximum drawdown
cumulative_max = jan_df['portfolio_1_close_px'].expanding().max()
drawdowns = (jan_df['portfolio_1_close_px'] - cumulative_max) / cumulative_max
max_drawdown = drawdowns.min()

print(f"January 2022 Portfolio Statistics:")
print(f"N Sample: {len(jan_df)}")
print(f"N Trade: {jan_df['signal_1'].abs().sum()}")
print(f"Simple Return: {simple_return:.2%}")
print(f"Monthly Volatility: {monthly_volatility:.2%}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")



In [ ]:
# Get January 2022 start and end timestamps
jan_2022_start = pd.Timestamp('2022-01-01').timestamp() * 1000  # Convert to milliseconds
jan_2022_end = pd.Timestamp('2022-02-01').timestamp() * 1000  # Convert to milliseconds


# Filter data for January 2022
jan_df = df[(df['1m_open_time'] >= jan_2022_start) & (df['1m_open_time'] < jan_2022_end)]
# Calculate monthly simple return using first and last close prices
first_close = jan_df['1m_close_px'].iloc[0]
last_close = jan_df['1m_close_px'].iloc[-1]
simple_return = (last_close - first_close) / first_close
# Calculate monthly volatility using daily returns
minutely_returns = jan_df['1m_close_px'].pct_change()
monthly_volatility = minutely_returns.std() * np.sqrt(len(jan_df))  # Monthly volatility
# Calculate maximum drawdown
cumulative_max = jan_df['1m_close_px'].expanding().max()
drawdowns = (jan_df['1m_close_px'] - cumulative_max) / cumulative_max
max_drawdown = drawdowns.min()

print(f"January 2022 BTC Statistics:")
print(f"N Sample: {len(jan_df)}")
print(f"Simple Return: {simple_return:.2%}")
print(f"Monthly Volatility: {monthly_volatility:.2%}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")


In [ ]:
def calculate_performance(df: pd.DataFrame, year: int, month: int) -> dict:
    # Validate month
    if not (1 <= month <= 12):
        raise ValueError(f"Invalid month: {month}. Month must be between 1 and 12.")

    # Check required columns
    required_columns = ['Date', 'Close', 'N_Trades']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise KeyError(f"Missing required column(s): {', '.join(missing_columns)}")

    # Get start and end timestamps in milliseconds
    start = pd.Timestamp(f"{year}-{month:02d}-01").timestamp() * 1000
    if month == 12:
        end = pd.Timestamp(f"{year + 1}-01-01").timestamp() * 1000
    else:
        end = pd.Timestamp(f"{year}-{month + 1:02d}-01").timestamp() * 1000

    # Filter data for the specified month
    monthly_df = df[(df['Date'] >= start) & (df['Date'] < end)]

    if monthly_df.empty:
        return {
            "n_sample": 0,
            "n_trade": 0,
            "monthly_return": 0.0,
            "monthly_volatility": 0.0,
            "monthly_mdd": 0.0
        }

    # Calculate simple return
    first_close = monthly_df['Close'].iloc[0]
    last_close = monthly_df['Close'].iloc[-1]
    simple_return = (last_close - first_close) / first_close

    # Calculate minutely returns and volatility
    minutely_returns = monthly_df['Close'].pct_change().dropna()
    monthly_volatility = minutely_returns.std() * np.sqrt(len(minutely_returns))

    # Calculate maximum drawdown
    cumulative_max = monthly_df['Close'].expanding().max()
    drawdowns = (monthly_df['Close'] - cumulative_max) / cumulative_max
    max_drawdown = drawdowns.min()

    n_trades = monthly_df['N_Trades'].sum()

    return {
        "n_sample": len(monthly_df),
        "n_trade": n_trades,
        "monthly_return": simple_return,
        "monthly_volatility": monthly_volatility,
        "monthly_mdd": max_drawdown
    }

In [ ]:

df_perf = df.rename(columns={"1m_open_time": "Date", "1m_close_px": "Close"})
df_perf["N_Trades"] = df["1m_number_of_trades"].abs()
res = calculate_performance(df_perf, 2022, 1)
print(f"January 2022 BTC Statistics:")
print(f"N Sample: {res['n_sample']}")
print(f"N Trade: {res['n_trade']}")
print(f"Simple Return: {res['monthly_return']:.2%}")
print(f"Monthly Volatility: {res['monthly_volatility']:.2%}")
print(f"Maximum Drawdown: {res['monthly_mdd']:.2%}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Prepare the dataframe
df_perf = df.rename(columns={"1m_open_time": "Date", "1m_close_px": "Close"})
df_perf["N_Trades"] = df["1m_number_of_trades"].abs()

# Collect statistics
year = 2022
returns = []
vols = []
mdds = []
annotations = []

for month in range(1, 13):
    try:
        res = calculate_performance(df_perf, year, month)
        ret = res['monthly_return']
        vol = res['monthly_volatility']
        mdd = res['monthly_mdd']

        returns.append(ret)
        vols.append(vol)
        mdds.append(mdd)

        annotation = f"{ret:.2%}\nσ={vol:.2%}\nDD={mdd:.2%}"
        annotations.append(annotation)
    except Exception as e:
        returns.append(np.nan)
        annotations.append("N/A")

# Convert data into 3x4 shape for heatmap
returns_matrix = np.array(returns).reshape(3, 4)
annot_matrix = np.array(annotations).reshape(3, 4)

# Plot heatmap with annotations
plt.figure(figsize=(10, 5))
sns.heatmap(
    returns_matrix,
    annot=annot_matrix,
    fmt='',
    cmap='RdYlGn',
    center=0,
    linewidths=0.5,
    cbar_kws={'label': 'Monthly Return'}
)

# Label setup
plt.title(f"{year} Monthly BTC Performance Heatmap")
plt.xticks(ticks=np.arange(4)+0.5, labels=["Jan–Mar", "Apr–Jun", "Jul–Sep", "Oct–Dec"])
plt.yticks(ticks=np.arange(3)+0.5, labels=["Row 1", "Row 2", "Row 3"], rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Prepare the dataframe
df_perf = df.rename(columns={"1m_open_time": "Date", "1m_close_px": "Close"})
df_perf["N_Trades"] = df["1m_number_of_trades"].abs()

# Collect statistics
year = 2023
returns = []
vols = []
mdds = []
annotations = []

for month in range(1, 13):
    try:
        res = calculate_performance(df_perf, year, month)
        ret = res['monthly_return']
        vol = res['monthly_volatility']
        mdd = res['monthly_mdd']

        returns.append(ret)
        vols.append(vol)
        mdds.append(mdd)

        annotation = f"{ret:.2%}\nσ={vol:.2%}\nDD={mdd:.2%}"
        annotations.append(annotation)
    except Exception as e:
        returns.append(np.nan)
        annotations.append("N/A")

# Convert data into 3x4 shape for heatmap
returns_matrix = np.array(returns).reshape(3, 4)
annot_matrix = np.array(annotations).reshape(3, 4)

# Plot heatmap with annotations
plt.figure(figsize=(10, 5))
sns.heatmap(
    returns_matrix,
    annot=annot_matrix,
    fmt='',
    cmap='RdYlGn',
    center=0,
    linewidths=0.5,
    cbar_kws={'label': 'Monthly Return'}
)

# Label setup
plt.title(f"{year} Monthly BTC Performance Heatmap")
plt.xticks(ticks=np.arange(4)+0.5, labels=["Jan–Mar", "Apr–Jun", "Jul–Sep", "Oct–Dec"])
plt.yticks(ticks=np.arange(3)+0.5, labels=["Row 1", "Row 2", "Row 3"], rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Create a copy of the filtered data to avoid SettingWithCopyWarning
df_hourly = df[df['is_1h_closed'] == True].copy()

# Calculate 30-period moving average using .loc to avoid warning
df_hourly.loc[:, '1h_ma_30'] = df_hourly['1h_close_px'].rolling(window=30).mean()
df_ma = pd.merge(df, df_hourly[['1m_open_time', '1h_ma_30']], on='1m_open_time', how='left')
df_ma_interpolated = df_ma.copy()
df_ma_interpolated['1h_ma_30'] = df_ma_interpolated['1h_ma_30'].ffill()
df_ma_interpolated.dropna(inplace=True)
df_ma_interpolated['1h_ma_30_diff'] = (df_ma_interpolated['1m_close_px'] - df_ma_interpolated['1h_ma_30']) / df_ma_interpolated['1h_ma_30']
# df_ma_interpolated['1h_ma_30_diff']
# Calculate quantiles
q_01 = df_ma_interpolated['1h_ma_30_diff'].quantile(0.01)
q_99 = df_ma_interpolated['1h_ma_30_diff'].quantile(0.99)

# Create histogram
plt.figure(figsize=(10, 6))
plt.hist(df_ma_interpolated['1h_ma_30_diff'], bins=100, density=True, alpha=0.7)

# Add vertical lines for quantiles
plt.axvline(q_01, color='r', linestyle='--', label=f'1% quantile: {q_01:.3f}')
plt.axvline(q_99, color='r', linestyle='--', label=f'99% quantile: {q_99:.3f}')

plt.title('Distribution of 1-Hour MA 30 Price Differences')
plt.xlabel('Price Difference from MA')
plt.ylabel('Density')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# df_hourly['signal_2'] = 

In [ ]:
df_ma_interpolated

In [ ]:
df_ma_interpolated['signal_2'] =  df_ma_interpolated['1h_ma_30_diff'] < q_01
# df_ma_interpolated['signal_3'] =  df_ma_interpolated['1h_ma_30_diff'] > q_99

n = len(df_ma_interpolated)
usdt_list = [0] * n
btc_list = [0] * n
portfolio_list = [0] * n

usdt_list[0] = 1000
btc_list[0] = 0
portfolio_list[0] = 1000
hold = 0
stop_loss = 0.03

n_trade_usdt = 500


    
df["btc_1"] = btc_list
df["usdt_1"] = usdt_list
df["portfolio_1_open_px"] = df["btc_1"] * df["1m_open_px"] + df["usdt_1"]
df["portfolio_1_high_px"] = df["btc_1"] * df["1m_high_px"] + df["usdt_1"]
df["portfolio_1_low_px"] = df["btc_1"] * df["1m_low_px"] + df["usdt_1"]
df["portfolio_1_close_px"] = df["btc_1"] * df["1m_close_px"] + df["usdt_1"]




In [ ]:
# Reset all previous calculations and start fresh
# Implementing Black-Scholes stock price simulation from scratch

import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Set simulation parameters
S0 = 100  # Initial stock price
mu = 0.1  # Expected annual return (10%)
sigma = 0.2  # Annual volatility (20%)
T = 1.0  # Time horizon in years
N = 252  # Number of trading days
dt = T/N  # Time step size

# Generate random normal numbers for Brownian motion
np.random.seed(42)
W = np.random.standard_normal(N)

# Initialize arrays
t = np.linspace(0, T, N)
S = np.zeros(N)

# Generate stock price path
S[0] = S0
for i in range(1, N):
    S[i] = S[i-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * W[i-1])

# Create timestamps for the simulation
base_date = datetime.now()
dates = [base_date + timedelta(days=x) for x in range(N)]

# Create DataFrame with simulated prices
df_simulated = pd.DataFrame({
    'Date': dates,
    'Price': S,
    'Returns': np.log(S[1:]/S[:-1]).tolist() + [0]  # Log returns
})

# Plot simulated price path
plt.figure(figsize=(12, 6))
plt.plot(df_simulated['Date'], df_simulated['Price'], label='Simulated Stock Price')
plt.title('Black-Scholes Stock Price Simulation')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Display summary statistics
print("\nSimulation Summary Statistics:")
print(df_simulated['Price'].describe())
print("\nDaily Returns Summary Statistics:")
print(df_simulated['Returns'].describe())



In [ ]:
df_simulated["SimpleReturn"] = df_simulated["Price"].pct_change()
df_simulated["CumulativeReturn"] = (1 + df_simulated["SimpleReturn"]).cumprod()
# df_simulated["CumulativeReturn"].plot()
# Plot simulated price path
plt.figure(figsize=(12, 6))
plt.plot(df_simulated['Date'], df_simulated['CumulativeReturn'], label='Simulated Stock Return')
plt.title('Black-Scholes Stock Price Simulation')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


